Library & Parameters 

In [ ]:
import numpy as np
from scipy.stats import norm
import datetime
from datetime import datetime


S0 = 100                      # Prix spot du sous-jacent
K = 100                       # Prix d'exercice (strike)
purchase_date = "2023-01-01"  # Date d'achat de l'option
maturity_date = "2024-01-01"  # Date d'échéance de l'option
#T = 1.0                      # Maturité en années
r = 0.05                      # Taux sans risque (5%)
sigma = 0.2                   # Volatilité annuelle (20%)
option_type = 'call'          # 'call' or 'put'
N = 10000                     # Nombre de simulations
n_steps = 10000               # Nombre de pas 
q = 0                         # Taux de dividende

purchase_date = datetime.strptime(purchase_date, "%Y-%m-%d")
maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")
T = (maturity_date - purchase_date).days / 365
print(f"Maturity in Years: {T}")

Maturity in Years: 1.0


In [114]:
# PRICER BLACK-SCHOLES

def black_scholes(S0, K, T, r, sigma, option_type):
    """Prix Black-Scholes-Merton pour option européenne sans dividendes."""
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == 'call':
        price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type.lower() == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("option_type doit être 'call' ou 'put'")
    
    return price

bs_price = black_scholes(S0, K, T, r, sigma, option_type)
print(f"Prix {option_type.capitalize()} (BSM) : {bs_price:.4f}")


Prix Call (BSM) : 10.4506


euro VO With no dividends of the underlying asset - Binomial Tree

In [115]:
# PRICER BINOMIAL

def binomial_tree_crr(S0, K, T, r, sigma, N, option_type='call'):
    """
    Prix d'une option européenne via arbre binomial Cox-Ross-Rubinstein (CRR).
    """
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    # Prix possibles du sous-jacent à maturité
    ST = np.array([S0 * (u**j) * (d**(N - j)) for j in range(N + 1)])
    
    # Payoff à maturité
    if option_type.lower() == 'call':
        option_values = np.maximum(ST - K, 0)
    elif option_type.lower() == 'put':
        option_values = np.maximum(K - ST, 0)
    else:
        raise ValueError("option_type doit être 'call' ou 'put'")
    
    # Remontée dans l'arbre
    for i in range(N - 1, -1, -1):
        option_values = np.exp(-r * dt) * (p * option_values[1:] + (1 - p) * option_values[:-1])
    
    return option_values[0]

bt_price = binomial_tree_crr(S0, K, T, r, sigma, N, option_type)
print(f"Prix {option_type.capitalize()} (Binomial Tree CRR, N={N}) : {bt_price:.4f}")

Prix Call (Binomial Tree CRR, N=10000) : 10.4504


euro VO With no dividends of the underlying asset - Monte Carlo

In [116]:
# PRICER MONTE CARLO

# Générer N valeurs aléatoires normales
Z = np.random.randn(N)

# Simuler le prix du sous-jacent à l'échéance (S_T)
S_T = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)

# Calculer le payoff
if option_type == 'call':
    payoffs = np.maximum(S_T - K, 0)
elif option_type == 'put':
    payoffs = np.maximum(K - S_T, 0)
else:
    raise ValueError("option_type doit être 'call' ou 'put'")

# Actualiser la moyenne des payoffs
option_price = np.exp(-r * T) * np.mean(payoffs)

print(f"Prix de l'option {option_type} par Monte Carlo : {option_price:.4f}")

Prix de l'option call par Monte Carlo : 10.3939


In [117]:
print(f"Prix {option_type.capitalize()} (BSM) : {bs_price:.10f}")
print(f"Prix {option_type.capitalize()} (Binomial Tree CRR, N={N}) : {bt_price:.10f}")
print(f"Prix de l'option {option_type} par Monte Carlo : {option_price:.10f}")

Prix Call (BSM) : 10.4505835722
Prix Call (Binomial Tree CRR, N=10000) : 10.4503836029
Prix de l'option call par Monte Carlo : 10.3939389950
